# Installing required libraries for Unsloth, Xformers, and other dependencies.


In [1]:
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton
!pip install transformers datasets

# Mounting Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Loading the saved model from drive

In [3]:
from transformers import AutoTokenizer
from peft import AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained(
    "/content/drive/MyDrive/Nebula9.ai",
    load_in_4bit = True,
)
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Nebula9.ai")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:174: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/165k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.31G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

# Example 1:

In [6]:
prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Based on the dietary tags and available ingredients, generate a detailed recipe that includes the following:
1. Recipe Name
2. Recipe Directions
3. Nutritional Values (Calories, Protein, Fat, Sodium)

Ensure that the recipe fits the dietary restrictions specified by the tags and make adjustments to the ingredients if necessary to meet those requirements.

### Input:
{}

### Response:
{}
"""

def generateRecipe(dietary_tag, ingredients):
    inputs = tokenizer(
        [
           prompt.format(f"Dietary tags: {dietary_tag}.\nAvailable ingredients: {ingredients}", "")
        ],
        return_tensors="pt"
    ).to("cuda")

    outputs = model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=2048,
        pad_token_id=tokenizer.eos_token_id
    )

    script = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    response = script[0].split("### Response:")[1].split("</s>")[0].strip()
    return response

dietary_tag = input("👉 Enter your Dietary Tag (e.g., vegan, low fat, high calorie): ")
ingredients = input("👉 Enter available ingredients (comma-separated, e.g., tomatoes, olive oil, salt): ")

print("\n🌟 Generating your custom recipe based on dietary preferences... Please wait! 🌟\n")

recipe = generateRecipe(dietary_tag, ingredients)

print("********************************************************")
print("                      🍽️ Recipe 🍽️                       ")
print("********************************************************")
print(recipe)
print("********************************************************")
print("        Enjoy your meal! 🥗 🍲 🍛 Bon Appétit! 🥗 🍲 🍛       ")
print("********************************************************")


👉 Enter your Dietary Tag (e.g., vegan, low fat, high calorie): low calorie
👉 Enter available ingredients (comma-separated, e.g., tomatoes, olive oil, salt): 1/2 cup slivered almonds, 1/4 cup olive oil, 1/2 tsp kosher salt

🌟 Generating your custom recipe based on dietary preferences... Please wait! 🌟

********************************************************
                      🍽️ Recipe 🍽️                       
********************************************************
Recipe Name: almond pesto .
Recipe Directions: ["Pulse almonds in a food processor until finely ground. Add oil and salt and pulse until a paste forms."].
Nutrition - Calories: 140.0 kcal, Protein: 2.0 g, Fat: 14.0 g, Sodium: 150.0 mg.
********************************************************
        Enjoy your meal! 🥗 🍲 🍛 Bon Appétit! 🥗 🍲 🍛       
********************************************************


# Example 2:

In [7]:
def generateRecipe(dietary_tag, ingredients):
    inputs = tokenizer(
        [
           prompt.format(f"Dietary tags: {dietary_tag}.\nAvailable ingredients: {ingredients}", "")
        ],
        return_tensors="pt"
    ).to("cuda")

    outputs = model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=2048,
        pad_token_id=tokenizer.eos_token_id
    )

    script = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    response = script[0].split("### Response:")[1].split("</s>")[0].strip()
    return response

dietary_tag = input("👉 Enter your Dietary Tag (e.g., vegan, low fat, high calorie): ")
ingredients = input("👉 Enter available ingredients (comma-separated, e.g., tomatoes, olive oil, salt): ")

print("\n🌟 Generating your custom recipe based on dietary preferences... Please wait! 🌟\n")

recipe = generateRecipe(dietary_tag, ingredients)

print("********************************************************")
print("                      🍽️ Recipe 🍽️                       ")
print("********************************************************")
print(recipe)
print("********************************************************")
print("        Enjoy your meal! 🥗 🍲 🍛 Bon Appétit! 🥗 🍲 🍛       ")
print("********************************************************")

👉 Enter your Dietary Tag (e.g., vegan, low fat, high calorie): vegan
👉 Enter available ingredients (comma-separated, e.g., tomatoes, olive oil, salt): apple, chives, mustard

🌟 Generating your custom recipe based on dietary preferences... Please wait! 🌟

********************************************************
                      🍽️ Recipe 🍽️                       
********************************************************
Recipe Name: apple chive mustard .
Recipe Directions: ["1. Peel and core the apple. Cut into chunks and place in a food processor.", "2. Add the chives and mustard. Pulse until smooth."].
Nutrition - Calories: 100.0 kcal, Protein: 1.0 g, Fat: 1.0 g, Sodium: 10.0 mg.
********************************************************
        Enjoy your meal! 🥗 🍲 🍛 Bon Appétit! 🥗 🍲 🍛       
********************************************************
